In [40]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Q1. Outlier가 데이터 셋에 exist할 경우 선형회귀에 대한 성능을 compare해보세요


# 데이터들의 의미
(https://www.kaggle.com/c/nyc-taxi-trip-duration/data)

id - a unique identifier for each trip

vendor_id - a code indicating the provider associated with the trip record

pickup_datetime - date and time when the meter was engaged

dropoff_datetime - date and time when the meter was disengaged

passenger_count - the number of passengers in the vehicle (driver entered value)

pickup_longitude - the longitude where the meter was engaged

pickup_latitude - the latitude where the meter was engaged

dropoff_longitude - the longitude where the meter was disengaged

dropoff_latitude - the latitude where the meter was disengaged

store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip



# target value: trip_duration - duration of the trip in seconds

In [3]:
data_df = pd.read_csv('1_train.csv')

In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


모든 feature가 수치형 데이터임이 아님을 알 수있다.

id, vendor_id, store_and_fwd_flag에 대한 처리가 필요!

In [30]:
# index가 id역할을 하니까 id는 드랍함
data_df.drop('id', axis=1, inplace=True)

In [31]:
np.round(data_df.describe(),2)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00,1458644.00
mean,1.53,1.66,-73.97,40.75,-73.97,40.75,959.49
std,0.50,1.31,0.07,0.03,0.07,0.04,5237.43
min,1.00,0.00,-121.93,34.36,-121.93,32.18,1.00
25%,1.00,1.00,-73.99,40.74,-73.99,40.74,397.00
50%,2.00,1.00,-73.98,40.75,-73.98,40.75,662.00
75%,2.00,2.00,-73.97,40.77,-73.96,40.77,1075.00
max,2.00,9.00,-61.34,51.88,-61.34,43.92,3526282.00


min, quantile, max를 살펴보면 어떤 outlier가 trip_duration에 있음을 알 수있습니다.

vendor id, id는 각각 identify를 위한 데이터이므로 drop을 사용하여 처리합니다. pick up time, drop off time은 각각 시계열 데이터인데, 이 책에서 시계열 처리에 대해 다루지 않았으므로 drop을 해줍니다.

In [15]:
data_df.drop(['pickup_datetime', 'dropoff_datetime'], axis = 1, inplace = True)

In [32]:
data_df.head(3)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,1,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,2,1,-73.979027,40.763939,-74.005333,40.710087,N,2124


store_and_fwd_flag와 trip_duration의 상관관계를 알아본다. 만약 상관관계가 적다면 drop하고 그렇지 않다면 핸들링을 따로 한다

In [13]:
sff_cat = {'N':0, 'Y':1}
cat_fun = lambda x:sff_cat[x]
sff = data_df['store_and_fwd_flag'].map(cat_fun)
data_df.groupby('store_and_fwd_flag')['trip_duration'].mean()

store_and_fwd_flag
N     958.819706
Y    1080.763331
Name: trip_duration, dtype: float64

위의 결과를 보고 store_and_fwd_flag을 drop할지 하지 않을지 생각해보세요

In [34]:
# N일 떄의 평균, Y일 때의 평균이 큰 차이가 없으므로 drop한다
data_df.drop('store_and_fwd_flag', axis=1, inplace=True)

In [35]:
data_df.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2,1,-73.982155,40.767937,-73.964630,40.765602,455
1,1,1,-73.980415,40.738564,-73.999481,40.731152,663
2,2,1,-73.979027,40.763939,-74.005333,40.710087,2124
3,2,1,-74.010040,40.719971,-74.012268,40.706718,429
4,2,1,-73.973053,40.793209,-73.972923,40.782520,435


trip_duration에 outlier 수를 파악해보고, 이 outlier들을 drop했을 때와 안했을 때의 성능 수준을 선형 회귀 모델들로 비교해 보겠습니다.

In [36]:
np.sum(data_df.trip_duration > 10000)

2123

# outlier가 있는 데이터셋을 사용하여 선형 회귀 모델 평가.

In [37]:
#outlier를 제거하지 않은 데이터 셋을 훈련세트와 데이터 세트로 나눈다.
X = data_df.drop('trip_duration', axis = 1)
y = data_df.trip_duration

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

LinearRegression을 사용하여 train set을 학습시킨후 test set으로 평가해보세요

In [41]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_preds = lr.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_preds)

print ("MAE: {0: 3f}, MSE: {1: 3f}, RMSE: {2: 3f}, R^2: {3: 3f}".format(mae, mse, rmse, r2))

MAE:  610.650508, MSE:  10602333.630458, RMSE:  3256.122484, R^2:  0.005634


Pipeline과 PolynomialFeatures를 가지고 다항회귀를 사용해서 평가해보세요

Ridge 모델 학습

다양한 learning rate 가지고 ridge 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

Lasso 모델 학습

다양한 learning rate 가지고 Lasso 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

ElasticNet 모델 학습

다양한 learning rate 가지고 ElasticNet 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

# outlier를 없엔 데이터 셋을 사용하여 선형 회귀모델 평가.

In [ ]:
outlier_idx = y[y > 10000].index
data_dropped_outlier = data_df.drop(outlier_idx, axis = 0)

In [ ]:
#데이터가 2000개 가량 줄어든 것을 확인할 수 있다.
data_dropped_outlier.info()

In [ ]:
X_dropped = data_dropped_outlier.drop('trip_duration', axis = 1)
y_dropped = data_dropped_outlier.trip_duration

X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, random_state = 42)

앞서서 했던 outlier를 제거하기 전 시행했던 각각의 모델들을 위의 데이터셋을 통해 평가해보세요.

# Q2. 이 데이터 셋에서는 규제가 그리 효과적이지 못한 것 처럼 보입니다. 그 이유는 무엇일까요?



# Q3. 이 데이터셋은 우리가 원하는 trip duration을 구하기에는 연관이 적은 것 같습니다. 데이터를 가공한다면, 어떤식으로 하면 trip duration과 관련이 있을까요?

# 부록:파이썬의 시계열 데이터 

파이썬 라이브러리에는 날짜와 시간을 위한 자료형과 달력과 관련된 기능을 제공하는 자료형이 있습니다..

파이썬 기본 자료형: datetime, time, calender

date: 그레고리언 달력을 사용하여 년, 월, 일을 저장한다.

time: 하루 중 시간을 시간, 분, 초, 마이크로 초 단위로 저장한다.

datetime: 날짜와 시간을 같이 저장한다

timedelta: 두 datetime 값 간의 차이(일, 초, 마이크로초)를 표현한다

In [ ]:
trip_df = pd.read_csv('./train.csv')
trip_df.drop(['id', 'vendor_id'], axis= 1,inplace = True)

간단한 시계열 포멧

%Y: 네자리 연도

%y: 두자리 연도

%m: 두자리 월 ex:01 은 1월

%d: 두자리 일 ex:03은 3일

%H: 24시간 형식 시간

%I: 12시간 형식 시간

%M: 두 자리 분

%S: 초

In [ ]:
from datetime import datetime
# series에 있는 시계열 데이터는 현재 문자열 형태로 저장되어 있기 때문에 처리가 용이한 datetime 타입으로 바꾼다.
d2dt = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S') #datetime형으로 변환할 때 시계열 데이터의 형태를 맞춰서 넣어줘야한다.
pickup = trip_df.pickup_datetime.map(d2dt)
dropoff = trip_df.dropoff_datetime.map(d2dt)

In [ ]:
#pickup 시간과 dropoff 시간의 간격을 초 단위로 바꾼다.
# datetime 형을 빼거나 더하면 timedelta 타입으로 바뀐다. timedelta 형은 시계열 형 데이터의 기간을 표현하는데 적합하다.
timegap = dropoff - pickup
tomsec = lambda x: x.seconds
timegap_sec = timegap.map(tosec)

In [ ]:
#timedelta형임을 확인할 수 있다.
type(timegap[0])

In [ ]:
timegap_sec

In [ ]:
#손님을 태운 시간과 내려준 시간의 차, 즉 trip_duration이 실제로는 이 데이터셋으로부터 바로 구할 수있다.
#위 데이터셋에서 trip_duration과 timegap_sec가 맞지 않는 4개의 데이터셋이 있는데 이는 시간이 잘못 기록되었음을 유추할 수 있다.
np.sum(timegap_sec != trip_df.trip_duration)